In [1]:
using PCT

[ Info: Precompiling PCT [ef708a43-c8a3-43f4-8f65-1f04ee4c5bb0]


In [2]:
f, ctx = @pct begin
    @space V begin
        type=(I,)->C
    end

    @space H begin
        type=(I,I)->C
    end
    (A::H) -> _
end

(Map(MapType(VecType(AbstractPCTType[MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict(:name => :H))]), UndeterminedPCTType(), Dict{Any, Any}()), PCTVector(VecType(AbstractPCTType[MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict(:name => :H))]), Var{MapType}[Var{MapType}(MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict(:name => :H)), :A)]), Var{UndeterminedPCTType}(UndeterminedPCTType(), :_)), TypeContext(Dict{Symbol, Vector{<:AbstractPCTType}}(:_ => AbstractPCTType[UndeterminedPCTType()], :H => AbstractPCTType[MapType(VecType(AbstractPCTType[I(), I()]), C(), Dict(:name => :H))], :V => AbstractPCTType[MapType(VecType(AbstractPCTType[I()]), C(), Dict(:name => :V))])))

In [3]:
g = fc(@pct f ctx (x::R) -> x)

(x) -> 
    x

In [4]:
cg = decompose(g)

x: 


In [5]:
as_map(cg)

(x) -> 
    x

In [6]:
eval_all(pp(cg))

(_z_0, _k_0) -> 
    _k_0

In [7]:
g = fc(@pct f ctx (x::R) -> 1 +  x*x)

(x) -> 
    (1+(x⋅x))

In [8]:
cg = decompose(g)

x: +1 ◀ *x


In [9]:
as_map(cg)

(x) -> 
    (1+(x⋅x))

In [10]:
pp(cg)

(_z_0, _k_0) -> 
    (∑((_i_0), (_i_0⋅δ(x, _i_0, _k_0)))+(x⋅_k_0))

In [11]:
eval_all(pp(cg))

(_z_0, _k_0) -> 
    (∑((_i_0), (_i_0⋅δ(x, _i_0, _k_0)))+(x⋅_k_0))

In [12]:
simplify(eval_all(pp(cg))) |> first

  0.120741 seconds (121.96 k allocations: 8.185 MiB, 99.64% compilation time)
1 swallow
(∑((_i_0), (_i_0⋅δ(x, _i_0, _k_0)))+(x⋅_k_0))
(∑((_i_0), δ(_i_0, x, (_i_0⋅_k_0)))+(x⋅_k_0))
-->

  0.016106 seconds (23.99 k allocations: 1.571 MiB, 99.70% compilation time)
2 contract_delta
(∑((_i_0), δ(_i_0, x, (_i_0⋅_k_0)))+(x⋅_k_0))
(2.0⋅x⋅_k_0)
-->

  0.000001 seconds


(_z_0, _k_0) -> 
    (2.0⋅x⋅_k_0)

In [13]:
g = fc(@pct f ctx (x::V) -> sum(i, x(i)))

(x) -> 
    ∑((i), x(i))

In [14]:
cg = decompose(g)

x: ∑i ◀ {i ⇥ x: ℳ x ◀ ->i}


In [15]:
eval_all(as_map(cg))

(x) -> 
    ∑((i), x(i))

In [16]:
pp(cg)

(_z_0, _k_0) -> 
    (_i_0) -> 
        ∑((i), δ(i, _i_0, _k_0))

In [17]:
simplify(pp(cg)) |> first

  0.012805 seconds (28.82 k allocations: 1.904 MiB, 99.64% compilation time)
1 contract_delta
∑((i), δ(i, _i_0, _k_0))
_k_0
-->

  0.000000 seconds


(_z_0, _k_0) -> 
    (_i_0) -> 
        _k_0

In [18]:
g = fc(@pct f ctx (x::V) -> sum(i, x(i)' * x(i)))

(x) -> 
    ∑((i), (x(i)'⋅x(i)))

In [19]:
cg = decompose(g)

x: ∑i ◀ {i ⇥ x: *x(i) ◀ † ◀ ℳ x ◀ ->i}


In [20]:
eval_all(as_map(cg))

(x) -> 
    ∑((i), (x(i)'⋅x(i)))

In [21]:
pp(cg)

(_z_0, _k_0) -> 
    (_i_1) -> 
        ∑((i), (δ(i, _i_1, (_k_0'⋅x(i)))+∑((_i_3), δ(i, _i_1, (_i_3'⋅δ(x(i)', _i_3, _k_0))))))

In [22]:
simplify(eval_all(pp(cg))) |> first

  0.111643 seconds (263.92 k allocations: 17.913 MiB, 6.57% gc time, 98.92% compilation time)
1 sum_dist
∑((i), (δ(i, _i_1, (_k_0'⋅x(i)))+∑((_i_3), δ(i, _i_1, (_i_3'⋅δ(x(i)', _i_3, _k_0))))))
(∑((i), δ(i, _i_1, (_k_0'⋅x(i))))+∑((i, _i_3), δ(i, _i_1, (_i_3'⋅δ(x(i)', _i_3, _k_0)))))
-->

  0.018354 seconds (79.01 k allocations: 5.265 MiB, 97.23% compilation time)
2 contract_delta
(∑((i), δ(i, _i_1, (_k_0'⋅x(i))))+∑((i, _i_3), δ(i, _i_1, (_i_3'⋅δ(x(i)', _i_3, _k_0)))))
(∑((i, _i_3), δ(i, _i_1, (_i_3'⋅δ(x(i)', _i_3, _k_0))))+(_k_0'⋅x(_i_1)))
-->

  0.000316 seconds (3.60 k allocations: 114.375 KiB)
3 contract_delta
(∑((i, _i_3), δ(i, _i_1, (_i_3'⋅δ(x(i)', _i_3, _k_0))))+(_k_0'⋅x(_i_1)))
(∑((_i_3), (_i_3'⋅δ(x(_i_1)', _i_3, _k_0)))+(_k_0'⋅x(_i_1)))
-->

  0.000198 seconds (2.20 k allocations: 69.156 KiB)
4 swallow
(∑((_i_3), (_i_3'⋅δ(x(_i_1)', _i_3, _k_0)))+(_k_0'⋅x(_i_1)))
(∑((_i_3), δ(_i_3, x(_i_1)', (_i_3'⋅_k_0)))+(_k_0'⋅x(_i_1)))
-->

  0.010656 seconds (35.78 k allocations: 2.446 MiB, 9

(_z_0, _k_0) -> 
    (_i_1) -> 
        ((_k_0'+_k_0)⋅x(_i_1))

In [23]:
g = fc(@pct f ctx (x::C) -> x' * x)

(x) -> 
    (x'⋅x)

In [24]:
cg = decompose(g)

x: *x ◀ †


In [25]:
pp(cg)

(_z_0, _k_0) -> 
    (∑((_i_0), (_i_0'⋅δ(x', _i_0, _k_0)))+(_k_0'⋅x))

In [28]:
simplify(pp(cg); settings=symmetry_settings) |> first

  0.000120 seconds (950 allocations: 30.109 KiB)
1 swallow
(∑((_i_0), (_i_0'⋅δ(x', _i_0, _k_0)))+(_k_0'⋅x))
(∑((_i_0), δ(_i_0, x', (_i_0'⋅_k_0)))+(_k_0'⋅x))
-->

  0.000014 seconds (8 allocations: 320 bytes)
2 contract_delta
(∑((_i_0), δ(_i_0, x', (_i_0'⋅_k_0)))+(_k_0'⋅x))
((_k_0'⋅x)+(x⋅_k_0))
-->

  0.000007 seconds (6 allocations: 256 bytes)
3 gcd
((_k_0'⋅x)+(x⋅_k_0))
(x⋅(_k_0'+_k_0))
-->

  0.000000 seconds


(_z_0, _k_0) -> 
    (x⋅(_k_0'+_k_0))